# Automatic Blog Generator

In this project, we will be  combining OpenAi with a document a page the lives on the internet(GitHub).

<h4>Setup Github Account</h4>
<ul>
    <li>Create a new public repository on github using the format: username.github.io</li>
 <li>Clone the repository</li>
 <li>Create an index.html file</li>
 <li>Add some content to the index file</li>
 <li>Push changes made and go to https://username.github.io to view document on the web</li>
    </ul>


In [2]:
import openai

In [3]:
from getpass import getpass
openai.api_key=getpass()

········


In [1]:
#Install the GitPython library. It allows us to make git calls within python code
!pip3 install GitPython

In [4]:
# Import Repo(Git repository) and set repo path as variables
from git import Repo

In [10]:
#Import Path functionality-This allows us to easily set paths
from pathlib import Path

In [6]:
pwd

'/Users/evelyn.nomayo/GitHub/project_gpt/gpt_env/etaNomayo.github.io'

In [14]:
PATH_TO_BLOG_REPO=Path('/Users/evelyn.nomayo/GitHub/project_gpt/gpt_env/etaNomayo.github.io/.git')

In [18]:
PATH_TO_BLOG_REPO.parent

PosixPath('/Users/evelyn.nomayo/GitHub/project_gpt/gpt_env/etaNomayo.github.io')

In [17]:
PATH_TO_BLOG=PATH_TO_BLOG_REPO.parent

In [19]:
#Create a sub directory called content
PATH_TO_BLOG/"content"

PosixPath('/Users/evelyn.nomayo/GitHub/project_gpt/gpt_env/etaNomayo.github.io/content')

In [20]:
#Asign a variable to the path to blog
PATH_TO_CONTENT= PATH_TO_BLOG/"content"

In [21]:
#There is actually no content folder yet, so lets create one
PATH_TO_CONTENT.mkdir(exist_ok=True, parents=True)

Now that we have setup our path, let us make sure we can use git to automate update and push changes onto the blog. So we will create a function for this.

In [22]:
# Function to update blog
def update_blog(commit_message='The Blog has just been updated'):
    #Sync to a reposotory using the gitPython library Repo
    repo=Repo(PATH_TO_BLOG_REPO)
    #Call the git add command to add new update vis the repo path
    repo.git.add(all=True)
    #git commit -m "The Blog has just been updated"
    repo.index.commit(commit_message)
    #git push
    origin = repo.remote(name='origin')
    origin.push()
    

In [23]:
#let us check if this works using a random text to uodate the blog post
randon_text_string ="Just testing here"

In [ ]:
with open(PATH_TO_BLOG/"index.html", 'w') as f:
    f